In these Experiments, we are using multiple types of voice data for PD detection. We perform three types of experiments,
1. without any data selection i.e. using all the three types of vowels together for each subject and utilizing all features.
2. selecting only features i.e. one dimensional data selection approsch. We consider the multiple types of samples for
   each subject and select features using chi2 test score. It will gives better performance than experiment 1.
3. To further improve PD detection accuracy, we propose a hypothesis that using mutiple types of voice data may degrade
   PD detection accuracy. That is selecting features using multiple types of samples is like obtaining one optimal subset of features for different types of voice samples or phonations. We propose a hypothesis that different types of voice samples might have different types of optimal subsets of features. Therefore, considering sample selection before feature selection might improve PD detection accuracy.
   Hence, our third experiment will use selection of samples, features and hyperparameters approach (SSFH method). The experimental results validate the hypothesis.

In [25]:
import numpy
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import KFold

import math
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import zscore
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
# fix random seed for reproducibility
numpy.random.seed(7)

In [26]:
#Importing the Train Data 
import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)
print(X[[1039]])

X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]
[[1.809000e+00 1.485100e-04 6.800000e-01 8.430000e-01 2.040000e+00
  7.881000e+00 7.820000e-01 2.690000e+00 4.543000e+00 1.107300e+01
  8.069000e+00 9.255540e-01 9.748100e-02 1.347200e+01 1.192600e+02
  1.216300e+02 8.028000e+00 1.081440e+02 1.375460e+02 6.200000e+01
  6.000000e+01 8.211245e-03 5.658130e-04 1.818200e+01 1.000000e+00
  3.387000e+00]]


In [27]:
i=2
indices = numpy.array(range(0, 3))
lb = 26
ub = 29
while i<=40:
    indices = numpy.append(indices, range(lb, ub))    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    lb = lb+26
    ub = ub+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
X_train_withlabels = X_withlabels[indices]
print(X_train_withlabels.shape)
Y_Vowels = X_train_withlabels[:, 26]
X_Vowels = numpy.delete(X_train_withlabels, 26, 1)
print(X_Vowels.shape)
print(Y_Vowels.shape)

(120,)
(120, 27)
(120, 26)
(120,)


# 1. Without any data selection (obtaining optimal performance by optimizing the network under LOSO cross validation)

In [4]:
Best_Acc=0
N = [1, 2, 3, 4, 5, 10,  12, 13, 15,  20, 23, 24, 25, 26, 30]
for n1 in N:
    for n2 in N:
        cor_class=0
        TP=0
        TN=0
        FP=0
        FN=0
        i=1
        lb=0
        ub=3
        cor_class=0
        while i<=40:

            Y_test = Y_Vowels[range(lb,ub)]
            Y_train = numpy.delete(Y_Vowels, range(lb,ub), 0)
            #Removing Labels

            X_test = X_Vowels[range(lb,ub)]
            #print(X_test.shape)
            X_train = numpy.delete(X_Vowels, range(lb,ub), 0)
            #print(X_train.shape)

            model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(n1, n2, ), random_state=1)
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)
            Acc = accuracy_score(Y_test, Y_pred)

            lb=lb+3
            ub=ub+3

            if(i<=20):
                #if numpy.sum(predictions)>13:
                if Acc>0.5:
                    cor_class = cor_class+1
                    TP = TP+1
            if(i>20):
                #if numpy.sum(predictions)<=13:
                if Acc>=0.5:
                    cor_class = cor_class+1
                    TN = TN+1

            i = i+1
        LOSO_Acc = (cor_class/40)*100
        if LOSO_Acc>=Best_Acc:
            Best_Acc = LOSO_Acc
            print("Average Accuracy of all the subjects, LOSO Acc ================", Best_Acc)
            print("Nodes1 =", n1, "Nodes2 =", n2)
            Sen = TP/20
            Spec = TN/20
            FP = 20-TN
            FN = 20-TP
            print("Sen ==================================", Sen*100)
            print("Spec =================================", Spec*100)
            print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 1
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 2
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 3
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 4
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 5
Sen ==================================

In [5]:
print("Done")

Done


# 2. Using only feature selection i.e., one dimensional data selection.

In [6]:
Best_Acc=0
K = range(1, 27)
for k in K:
    X_FS = SelectKBest(chi2, k=k).fit_transform(X_Vowels, Y_Vowels)
    print("Size of selected subset of features =", X_FS.shape)
    N = range(1, 31)
    for n1 in N:
        for n2 in N:
            cor_class=0
            TP=0
            TN=0
            FP=0
            FN=0
            i=1
            lb=0
            ub=3
            cor_class=0
            while i<=40:

                Y_test = Y_Vowels[range(lb,ub)]
                Y_train = numpy.delete(Y_Vowels, range(lb,ub), 0)
                #Removing Labels

                X_test = X_FS[range(lb,ub)]
                #print(X_test.shape)
                X_train = numpy.delete(X_FS, range(lb,ub), 0)
                #print(X_train.shape)

                model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(n1, n2, ), random_state=1)
                model.fit(X_train, Y_train)
                Y_pred = model.predict(X_test)
                Acc = accuracy_score(Y_test, Y_pred)

                lb=lb+3
                ub=ub+3

                if(i<=20):
                    #if numpy.sum(predictions)>13:
                    if Acc>0.5:
                        cor_class = cor_class+1
                        TP = TP+1
                if(i>20):
                    #if numpy.sum(predictions)<=13:
                    if Acc>=0.5:
                        cor_class = cor_class+1
                        TN = TN+1

                i = i+1
            LOSO_Acc = (cor_class/40)*100
            if LOSO_Acc>=Best_Acc:
                Best_Acc = LOSO_Acc
                print("Average Accuracy of all the subjects, LOSO Acc ================", Best_Acc)
                print("Nodes1 =", n1, "Nodes2 =", n2)
                Sen = TP/20
                Spec = TN/20
                FP = 20-TN
                FN = 20-TP
                print("Sen ==================================", Sen*100)
                print("Spec =================================", Spec*100)
                print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Size of selected subset of features = (120, 1)
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 1
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 2
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 3
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 Nodes2 = 4
Sen ================================== 0.0
Spec ================================= 0.0
Mathew Correlation Coefficients MCC= -1.0
Average Accuracy of all the subjects, LOSO Acc ================ 0.0
Nodes1 = 1 No

# 3. Two dimensional data selection approach i.e., selection of samples, features and hyperparameters (SSFH).

In [7]:
#This function will do selection of features and hyperparameters
def evaluate(X_Data, Y_data):
    ##############################Feature Selection Part#################
    
    K = range(1, 27)
    for k in K:
        #print("Selected Features =", k)
        X_FS = SelectKBest(chi2, k=k).fit_transform(X_Data, Y_data)
        Best_Acc =0
        print("Size of selected features ==============", X_FS.shape)
        ########### LOSO ##############################
        Nodes = range(1, 31)
        for nodes1 in Nodes:
            for nodes2 in Nodes:
                i = 0
                cor_class=0
                LOSO_Acc=0
                while i<=39:
                    X_test = X_FS[[i]]
                    X_train = numpy.delete(X_FS, i, 0)
                    Y_test = Y_data[[i]]
                    Y_train = numpy.delete(Y_data, i, 0)
                    ###############For Each Selected Features, evaluate model performance by trying different nodes############
                    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(nodes1, nodes2), random_state=1)
                    model.fit(X_train, Y_train)
                    predictions = model.predict(X_test)
                    if(i<=19):
                        if predictions[0]==1:
                            cor_class = cor_class+1
                    if(i>19):
                        if predictions[0]==0:
                            cor_class = cor_class+1

                    i = i+1
                LOSO_Acc= (cor_class/40)*100
                if (LOSO_Acc>Best_Acc):
                    Best_Acc = LOSO_Acc
                    print("Best Acc ====================================", Best_Acc)
                    print("Nodes1 =", nodes1, "Nodes2=", nodes2, "No of Selected Features =", k)

In [8]:
#This part is doing selection of samples. j=0 means S1 in the paper, j=1 means S2 of the paper and j=2 means S3 of the paper.
j = 0
while j<=2:
    print("This is Dataset No=======================================================================", j)
    i=1
    indices = j
    increment = 26+j
    while i<=39:

        indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
        i = i+1
        increment = increment+26
    print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
    #print(indices)
    X_train_withlabels = X_withlabels[[indices]]
    print(X_train_withlabels.shape)
    Y_New = X_train_withlabels[:, 26]
    X_New = numpy.delete(X_train_withlabels, 26, 1)
    print(indices)
    print(X_New.shape)
    print(Y_New.shape)
    print(Y_New)
    evaluate(X_New, Y_New)
    #evaluate_SVM_Linear(X_New, Y_New)
    j = j+1

This is Dataset No======================================================================= 0
(40,)
(40, 27)
[   0   26   52   78  104  130  156  182  208  234  260  286  312  338
  364  390  416  442  468  494  520  546  572  598  624  650  676  702
  728  754  780  806  832  858  884  910  936  962  988 1014]
(40, 26)
(40,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Size of selected features ============== (40, 1)
Best Acc ==================================== 22.5
Nodes1 = 2 Nodes2= 6 No of Selected Features = 1
Best Acc ==================================== 40.0
Nodes1 = 2 Nodes2= 7 No of Selected Features = 1
Best Acc ==================================== 45.0
Nodes1 = 5 Nodes2= 26 No of Selected Features = 1
Best Acc ==================================== 50.0
Nodes1 = 11 Nodes2= 15 No of Selected Features = 1
Best Acc ==================================== 62.5
Nodes1 = 17 Nodes2= 17 No of Selected Features =

In [9]:
print("Done")

Done


# The Above Algorithms will take time as they uses simple search strategies, so to regenerate the best performance for of the three experiment, we use the below given code.

# Experiment 1. No data selection.

In [28]:
Best_Acc=0

cor_class=0
TP=0
TN=0
FP=0
FN=0
i=1
lb=0
ub=3
cor_class=0
while i<=40:

    Y_test = Y_Vowels[range(lb,ub)]
    Y_train = numpy.delete(Y_Vowels, range(lb,ub), 0)
    #Removing Labels

    X_test = X_Vowels[range(lb,ub)]
    #print(X_test.shape)
    X_train = numpy.delete(X_Vowels, range(lb,ub), 0)
    #print(X_train.shape)

    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15, 10, ), random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    Acc = accuracy_score(Y_test, Y_pred)

    lb=lb+3
    ub=ub+3

    if(i<=20):
        #if numpy.sum(predictions)>13:
        if Acc>0.5:
            cor_class = cor_class+1
            TP = TP+1
    if(i>20):
        #if numpy.sum(predictions)<=13:
        if Acc>=0.5:
            cor_class = cor_class+1
            TN = TN+1

    i = i+1
LOSO_Acc = (cor_class/40)*100
if LOSO_Acc>=Best_Acc:
    Best_Acc = LOSO_Acc
    print("Average Accuracy of all the subjects, LOSO Acc ================", Best_Acc)
    print("Nodes1 =", n1, "Nodes2 =", n2)
    Sen = TP/20
    Spec = TN/20
    FP = 20-TN
    FN = 20-TP
    print("Sen ==================================", Sen*100)
    print("Spec =================================", Spec*100)
    print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Average Accuracy of all the subjects, LOSO Acc ================ 75.0
Nodes1 = 30 Nodes2 = 30
Sen ================================== 80.0
Spec ================================= 70.0
Mathew Correlation Coefficients MCC= 0.502518907629606


# Experiment No 2. Best Performance using feature selection only i.e., one dimensional data selection approach. Here we use multiple types of samples per subject and search optimal features.

In [29]:
Best_Acc=0

X_FS = SelectKBest(chi2, k=20).fit_transform(X_Vowels, Y_Vowels)
print("Size of selected subset of features =", X_FS.shape)
    
cor_class=0
TP=0
TN=0
FP=0
FN=0
i=1
lb=0
ub=3
cor_class=0
while i<=40:

    Y_test = Y_Vowels[range(lb,ub)]
    Y_train = numpy.delete(Y_Vowels, range(lb,ub), 0)
    #Removing Labels

    X_test = X_FS[range(lb,ub)]
    #print(X_test.shape)
    X_train = numpy.delete(X_FS, range(lb,ub), 0)
    #print(X_train.shape)

    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 30, ), random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    Acc = accuracy_score(Y_test, Y_pred)

    lb=lb+3
    ub=ub+3

    if(i<=20):
        #if numpy.sum(predictions)>13:
        if Acc>0.5:
            cor_class = cor_class+1
            TP = TP+1
    if(i>20):
        #if numpy.sum(predictions)<=13:
        if Acc>=0.5:
            cor_class = cor_class+1
            TN = TN+1

    i = i+1
LOSO_Acc = (cor_class/40)*100
if LOSO_Acc>=Best_Acc:
    Best_Acc = LOSO_Acc
    print("Average Accuracy of all the subjects, LOSO Acc ================", Best_Acc)
    print("Nodes1 =", n1, "Nodes2 =", n2)
    Sen = TP/20
    Spec = TN/20
    FP = 20-TN
    FN = 20-TP
    print("Sen ==================================", Sen*100)
    print("Spec =================================", Spec*100)
    print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))

Size of selected subset of features = (120, 20)
Average Accuracy of all the subjects, LOSO Acc ================ 87.5
Nodes1 = 30 Nodes2 = 30
Sen ================================== 100.0
Spec ================================= 75.0
Mathew Correlation Coefficients MCC= 0.7745966692414834


# Experiment No 3. Results obtained using the proposed 2-Dimensional Data Selection Approach (i.e., selecting samples before features selection) or simultaneous sample and feature selection.  Part 1. Results for S1 where S1 means we select vowel a sample for each subject. S1 means vowel a. S2 means vowel o and S3 means vowel u.

In [30]:
#This part is doing selection of samples. j=0 means S1 in the paper, i.e., vowel a sample for each subject is selected before feature selection.
j = 0

i=1
indices = j
increment = 26+j
while i<=39:

    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y_New = X_train_withlabels[:, 26]
X_New = numpy.delete(X_train_withlabels, 26, 1)
print(indices)
print(X_New.shape)
print(Y_New.shape)
print(Y_New)

(40,)
(40, 27)
[   0   26   52   78  104  130  156  182  208  234  260  286  312  338
  364  390  416  442  468  494  520  546  572  598  624  650  676  702
  728  754  780  806  832  858  884  910  936  962  988 1014]
(40, 26)
(40,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [31]:
X_FS = SelectKBest(chi2, k=4).fit_transform(X_New, Y_New)
Best_Acc =0
print("Size of selected features ==============", X_FS.shape)
########### LOSO ##############################

i = 0
TP=0
TN=0
FP=0
FN=0
cor_class=0
LOSO_Acc=0
while i<=39:
    X_test = X_FS[[i]]
    X_train = numpy.delete(X_FS, i, 0)
    Y_test = Y_New[[i]]
    Y_train = numpy.delete(Y_New, i, 0)
    ###############For Each Selected Features, evaluate model performance by trying different nodes############
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(4, 30), random_state=1)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    if(i<=19):
        if predictions==1:
            cor_class = cor_class+1
            TP = TP+1
    if(i>19):
        if predictions==0:
            cor_class = cor_class+1
            TN = TN+1

    i = i+1
LOSO_Acc= (cor_class/40)*100
if (LOSO_Acc>Best_Acc):
    Best_Acc = LOSO_Acc
    print("Best Acc ====================================", Best_Acc)
    Sen = TP/20
    Spec = TN/20
    FP = 20-TN
    FN = 20-TP
    print("Sen ==================================", Sen*100)
    print("Spec =================================", Spec*100)
    print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))
    

Size of selected features ============== (40, 4)
Best Acc ==================================== 80.0
Sen ================================== 85.0
Spec ================================= 75.0
Mathew Correlation Coefficients MCC= 0.6030226891555273


# Experiment 3, Part 2, Best Performance when S2 (i.e. vowel o) sample for each subject is selected.

In [32]:
#This part is doing selection of samples. j=1 means S2 in the paper i.e., vowel o sample for each subject is selected before feature selection.
j = 1

i=1
indices = j
increment = 26+j
while i<=39:

    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y_New = X_train_withlabels[:, 26]
X_New = numpy.delete(X_train_withlabels, 26, 1)
print(indices)
print(X_New.shape)
print(Y_New.shape)
print(Y_New)

(40,)
(40, 27)
[   1   27   53   79  105  131  157  183  209  235  261  287  313  339
  365  391  417  443  469  495  521  547  573  599  625  651  677  703
  729  755  781  807  833  859  885  911  937  963  989 1015]
(40, 26)
(40,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
X_FS = SelectKBest(chi2, k=3).fit_transform(X_New, Y_New)
Best_Acc =0
print("Size of selected features ==============", X_FS.shape)
########### LOSO ##############################

i = 0
TP=0
TN=0
FP=0
FN=0
cor_class=0
LOSO_Acc=0
while i<=39:
    X_test = X_FS[[i]]
    X_train = numpy.delete(X_FS, i, 0)
    Y_test = Y_New[[i]]
    Y_train = numpy.delete(Y_New, i, 0)
    ###############For Each Selected Features, evaluate model performance by trying different nodes############
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 24), random_state=1)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    if(i<=19):
        if predictions==1:
            cor_class = cor_class+1
            TP = TP+1
    if(i>19):
        if predictions==0:
            cor_class = cor_class+1
            TN = TN+1

    i = i+1
LOSO_Acc= (cor_class/40)*100
if (LOSO_Acc>Best_Acc):
    Best_Acc = LOSO_Acc
    print("Best Acc ====================================", Best_Acc)
    Sen = TP/20
    Spec = TN/20
    FP = 20-TN
    FN = 20-TP
    print("Sen ==================================", Sen*100)
    print("Spec =================================", Spec*100)
    print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))
    

Size of selected features ============== (40, 3)
Best Acc ==================================== 97.5
Sen ================================== 100.0
Spec ================================= 95.0
Mathew Correlation Coefficients MCC= 0.9511897312113419


# Experiment 3, Part 3, Best Performance when S3 (i.e. vowel u) sample for each subject is selected.

In [34]:
#This part is doing selection of samples. j=2 means S3 in the paper i.e., vowel u sample for each subject is selected before feature selection.
j = 2

i=1
indices = j
increment = 26+j
while i<=39:

    indices = numpy.append(indices, increment)    #Indices will hold all indices of ist three samples in each subject
    i = i+1
    increment = increment+26
print(indices.shape)         #Thus we get 40x3=120 indices of 120samples. Now extract those samples from X and Y to get a new Training Set
#print(indices)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y_New = X_train_withlabels[:, 26]
X_New = numpy.delete(X_train_withlabels, 26, 1)
print(indices)
print(X_New.shape)
print(Y_New.shape)
print(Y_New)

(40,)
(40, 27)
[   2   28   54   80  106  132  158  184  210  236  262  288  314  340
  366  392  418  444  470  496  522  548  574  600  626  652  678  704
  730  756  782  808  834  860  886  912  938  964  990 1016]
(40, 26)
(40,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [35]:
X_FS = SelectKBest(chi2, k=15).fit_transform(X_New, Y_New)
Best_Acc =0
print("Size of selected features ==============", X_FS.shape)
########### LOSO ##############################

i = 0
TP=0
TN=0
FP=0
FN=0
cor_class=0
LOSO_Acc=0
while i<=39:
    X_test = X_FS[[i]]
    X_train = numpy.delete(X_FS, i, 0)
    Y_test = Y_New[[i]]
    Y_train = numpy.delete(Y_New, i, 0)
    ###############For Each Selected Features, evaluate model performance by trying different nodes############
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(1, 23), random_state=1)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    if(i<=19):
        if predictions==1:
            cor_class = cor_class+1
            TP = TP+1
    if(i>19):
        if predictions==0:
            cor_class = cor_class+1
            TN = TN+1

    i = i+1
LOSO_Acc= (cor_class/40)*100
if (LOSO_Acc>Best_Acc):
    Best_Acc = LOSO_Acc
    print("Best Acc ====================================", Best_Acc)
    Sen = TP/20
    Spec = TN/20
    FP = 20-TN
    FN = 20-TP
    print("Sen ==================================", Sen*100)
    print("Spec =================================", Spec*100)
    print("Mathew Correlation Coefficients MCC=", (TP*TN-FP*FN)/(math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))))
    

Size of selected features ============== (40, 15)
Best Acc ==================================== 97.5
Sen ================================== 100.0
Spec ================================= 95.0
Mathew Correlation Coefficients MCC= 0.9511897312113419


# The Above Experiments, validate the hypothesis that multiple types of samples degrade the PD detection performance and improved performance can be obtained by utilizing sample selection before feature selection.

# Extended Experiment using LOSO on testing database.

In [40]:
#Plz add a row at the first row of the train_data and test_data file (available puclicly) that 
#will gives information about the features. The row entries for the training database will be as follows:
# F0, F1, F2 ........, F27, Y, where Y is the lable.
# The row entries for the testing database will be as follows:
# F0, F1, F2 ........, F26, Y, where Y is the lable. This is necessary because we need to select features in 
#the training database using the chi2 model, and then select the same subset of features in testing database.

import pandas as pd
df = pd.read_csv("E:\PD Dataset From Laptop/train_data.txt")
print(df.columns)
X_first = numpy.array(df)
X_train = numpy.delete(X_first, 27, 1)
X_withlabels = numpy.delete(X_train, 0,1)
print("X_with contains 26 input features and class label but not with patient id", X_withlabels.shape)               
X = numpy.delete(X_withlabels, 26, 1)
print("X contains 26 input features only and there is no output label", X.shape)
Y = X_withlabels[: ,26]
print(Y)

##########for the purpose of feature selection##########
df = df.drop('F0', 1)
df = df.drop('F27', 1)
df_X_train = df.drop('Y', 1)
print(df_X_train.columns)

Index(['F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', ' F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'Y'],
      dtype='object')
X_with contains 26 input features and class label but not with patient id (1040, 27)
X contains 26 input features only and there is no output label (1040, 26)
[1. 1. 1. ... 0. 0. 0.]
Index(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', ' F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26'],
      dtype='object')


In [41]:
i=2
indices = numpy.array(range(0, 2))
lb = 26
ub = 28
while i<=40:
    indices = numpy.append(indices, range(lb, ub))
    i = i+1
    lb = lb+26
    ub = ub+26
print(indices.shape)
X_train_withlabels = X_withlabels[[indices]]
print(X_train_withlabels.shape)
Y_train = X_train_withlabels[:, 26]
X_train = numpy.delete(X_train_withlabels, 26, 1)
print(X_train.shape)
print(Y_train.shape)
print(Y_train)

(80,)
(80, 27)
(80, 26)
(80,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [42]:
#Importing the Test Database 

df_test = pd.read_csv("E:\PD Dataset From Laptop/test_data.txt")
X_first = numpy.array(df_test)
X_test_withlabels = numpy.delete(X_first, 0,1)
X_test = numpy.delete(X_test_withlabels, 26, 1)
print("X_with contains 26 input features and class label but not with patient id", X_test_withlabels.shape)   
print(X_test.shape)
Y_test = X_test_withlabels[:, 26]
print(Y_test)

##############for the purpose of feature selection#############
##########for the purpose of feature selection##########
df_test = df_test.drop('F0', 1)
df_X_test = df_test.drop('Y', 1)
print(df_X_test.columns)

X_with contains 26 input features and class label but not with patient id (168, 27)
(168, 26)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Index(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', ' F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26'],
      dtype='object')


In [43]:
#This experiment is not important. But, some of previous methods have used this experiment in their papers,
#that is why we are also simulating it. 
from sklearn.feature_selection import SelectFromModel
Best_Acc=0

selector=SelectKBest(score_func=chi2,k=4)
X_train_FS = SelectKBest(chi2, k=4).fit_transform(X_train, Y_train)
print("Size of X_train =", X_train_FS.shape)
    
selector.fit(X_train,Y_train)
S = selector.get_support(True)
print(S)
X_test_FS = X_test[:, S]
print("Size of X_test_FS =", X_test_FS.shape)
   
i=1
correct_sub_classified = 0
lb=0
ub=6
while i<=28:


    #########Setting Train Test Parts for Each Subject
    X_test_subj = X_test_FS[[range(lb,ub)]]
    Y_test_subj = Y_test[[range(lb, ub)]]

    ##############Models Part##############################################
    model =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, ), random_state=1)
    model.fit(X_train_FS, Y_train)
    Y_pred = model.predict(X_test_subj)
    scores = accuracy_score(Y_test_subj, Y_pred)

    if scores>0.5:
        correct_sub_classified = correct_sub_classified +1

    lb=lb+6
    ub=ub+6
    i= i+1
Acc = (correct_sub_classified/28)*100
if (Acc>=Best_Acc):
    Best_Acc=Acc
    print("Best Acc ===========", Best_Acc)


Size of X_train = (80, 4)
[16 18 19 20]
Size of X_test_FS = (168, 4)
Best Acc =========== 100.0
